In [ ]:
import pandas as pd
import os
import re
from tqdm import tqdm
import pyodbc
import datetime
import cx_Oracle
from scipy import stats
from datetime import timedelta
import matplotlib.pyplot as plt
import numpy as np
from isoweek import Week
import json

In [ ]:
from statsmodels.stats.weightstats import zconfint

In [ ]:
server = ''
database = ''
username = ''
password = ''
driver= 'SQL Server'
connection = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)

requestString_sessions = '''
...
    '''

df = pd.read_sql_query(requestString_sessions,connection)

In [ ]:
df.head(1)

,SessionID,MessageID,ExpGroup,TimeComplete,PhraseText,IntentName
0,5cc5632c-e403-11ec-b9c5-ac100f8e0000,38adba08-cb73-4b95-a5ab-69aa43f04132,0.0,2022-06-04 15:39:23.740,Меня интересует в Азербайджан,default_qestion_faq


In [ ]:
df['thanks'] = list(df.PhraseText.str.contains('Спасибо|спасибо|спасибочки|спс|Благодарю|благодарю'))
df['thanks_rank'] =0
for i in range(len(df)):
    if df['thanks'].iloc[i]==True:
        df['thanks_rank'].iloc[i]=1
    else:
        df['thanks_rank'].iloc[i]=0

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
df['operator'] = list(df.IntentName.str.contains('employee_transfer|support_chat'))
df['operator_rank'] =0
for i in range(len(df)):
    if df['operator'].iloc[i]==True:
        df['operator_rank'].iloc[i]=1
    else:
        df['operator_rank'].iloc[i]=0

In [ ]:
sess_info=df[['SessionID','operator_rank', 'thanks_rank', 'ExpGroup']].groupby(['SessionID'], as_index=False).max()

In [ ]:
sess_info_0 = sess_info[sess_info.ExpGroup==0.0]

count_0 = len(sess_info_0)
sum_operators_0 = sum(sess_info_0.operator_rank)
sum_thanks_0 = sum(sess_info_0.thanks_rank)
avg_operators_0 = sum_operators_0/count_0
avg_operators_zconfint_0 = zconfint(sess_info_0.operator_rank)
avg_thanks_0 = sum_thanks_0/count_0
avg_thanks_zconfint_0 = zconfint(sess_info_0.thanks_rank)

sess_info_1 = sess_info[sess_info.ExpGroup==1.0]

count_1 = len(sess_info_1)
sum_operators_1 = sum(sess_info_1.operator_rank)
sum_thanks_1 = sum(sess_info_1.thanks_rank)
avg_operators_1 = sum_operators_1/count_1
avg_operators_zconfint_1 = zconfint(sess_info_1.operator_rank)
avg_thanks_1 = sum_thanks_1/count_1
avg_thanks_zconfint_1 = zconfint(sess_info_1.thanks_rank)


print('% сессий с вызовом оператора:', '\n',
      'Модель А: '+"{:.4f}".format(avg_operators_0)+' , доверительный интервал: '+str(avg_operators_zconfint_0), '\n'
      'Модель B: '+"{:.4f}".format(avg_operators_1)+' , доверительный интервал: '+str(avg_operators_zconfint_1), '\n\n'
      '% сессий с благодарностью:', '\n',
      'Модель А: '+"{:.4f}".format(avg_thanks_0)+' , доверительный интервал: '+str(avg_thanks_zconfint_0), '\n'
      'Модель B: '+"{:.4f}".format(avg_thanks_1)+' , доверительный интервал: '+str(avg_thanks_zconfint_1), '\n\n'
      )

% сессий с вызовом оператора: 
 Модель А: 0.1549 , доверительный интервал: (0.149353568217134, 0.16044699461169262) 
Модель B: 0.1415 , доверительный интервал: (0.13607623629987622, 0.14690823789711918) 

% сессий с благодарностью: 
 Модель А: 0.0329 , доверительный интервал: (0.030119427261331173, 0.035584720542412865) 
Модель B: 0.0348 , доверительный интервал: (0.031913778281448466, 0.03760661897796444) 


